In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
installs = pd.read_csv('../datos/installs.csv', dtype ={'click_hash': object, 'wifi': object, 'trans_id': object})

### Analizo columna por columna

In [3]:
# No tiene sentido quedarse con el pais, porque es uno solo
#No tiene sentido quedarse con la columna trans_id. El 98% de los valores son Null
#No tiene sentido quedarse con click_hash, el 99% de los valors son null
#No tiene sentido quedarse con event_uuid, el 78% de los valores son null
#No tiene sentido quedarse con kind, el 78% de los valores son null
#No tiene sentido quedarse con wifi, el 48% de los valores son null
#No tiene sentido quedarse con device_brand. El 42% son null
#No tiene sentido quedarse con la columna user_agent El 31% de los valores son Null
#¿Tiene sentido quedarse con la columna session_user_agent? El 3% de los valores son Null
# session_user_agent tiene el 3% de los valores null. Borro esas filas
del installs['trans_id']
del installs['device_countrycode']
del installs['click_hash']
del installs['event_uuid']
del installs['kind']
del installs['wifi']
del installs['device_brand']
del installs['user_agent']
del installs['session_user_agent']

##Borro las filas que tienen null en device_model y device_language. Juntas son el 5% de las filas en null
installs = installs[pd.notnull(installs['device_model'])]
installs = installs[pd.notnull(installs['device_language'])]

In [4]:
installs.dtypes

created             object
application_id       int64
ref_type             int64
ref_hash             int64
attributed            bool
implicit              bool
device_model       float64
ip_address           int64
device_language    float64
dtype: object

In [5]:
installs.count()

created            453354
application_id     453354
ref_type           453354
ref_hash           453354
attributed         453354
implicit           453354
device_model       453354
ip_address         453354
device_language    453354
dtype: int64

In [6]:
installs['created'] = pd.to_datetime(installs['created'], errors = 'coerce')

In [7]:
installs = installs.sort_values(by = ['ref_hash', 'created'])

In [8]:
events = pd.read_csv('../datos/events.csv',
                     dtype={
                         'device_countrycode': 'category', 
                         'device_city': 'category',
                         'ref_type': 'int64',
                         'application_id': 'int64',
                         'device_os_version': 'category',
                         'device_brand': 'category',
                         'device_model': 'float64',
                         'session_user_agent': 'category',
                         'trans_id': 'category',
                         'user_agent': 'category',
                         'carrier': 'category',
                         'device_os': 'category',
                         'device_os_version': 'category',
                         'device_language': 'category',
                         'connection_type': 'category',
                         'wifi': 'bool'
                     })

In [9]:
del events['device_countrycode'] #Borrada porque hay un solo país
del events['trans_id'] #Borrada por tener muy pocos valores (37000/7000000)
del events['event_uuid'] #Borrada por tener todos los valores distintos
del events['device_os_version'] #Borrado por tener 70% de los valores nulos.
del events['device_brand'] #Borrado por tener 67% de los valores nulos
del events['device_city'] #Borrado por tener 76% de los valores nulos
del events['user_agent'] #Borrado por tener 57% de los valores nulos
del events['carrier'] #Borrado por tener 75% de los valores nulos
del events['device_os'] #Borrado por tener 76% de los valores nulos
del events['connection_type'] #Borrado por tener 77% de los valores nulos
#------------------------------------------------------------------
events = events[pd.notnull(events['session_user_agent'])] #Borrados las filas nulas (0,5%)
events = events[pd.notnull(events['kind'])] #Borrados las filas nulas (0,5%)
#------------------------------------------------------------------
events = events[pd.notnull(events['device_model'])] #Borrado por tener 27% de nulos, podria rellenarse si no andan los algoritmos

In [10]:
del events['attributed']

In [ ]:
events['ref_type'] = events['ref_type'].as_type('float32')

In [11]:
clicks = pd.read_csv("../datos/clicks.csv", low_memory=False)

In [191]:
clicks["latitude"] = clicks["latitude"].astype("float32")
clicks["longitude"] = clicks["longitude"].astype("float32")
clicks["carrier_id"] = clicks["carrier_id"].astype("category")
clicks["timeToClick"] = clicks["timeToClick"].astype("float32")
clicks["touchX"] = clicks["touchX"].astype("float32")
clicks["touchY"] = clicks["touchY"].astype("float32")
clicks["source_id"] = clicks["source_id"].astype("category")
clicks["advertiser_id"] = clicks["advertiser_id"].astype("category") 

In [12]:
del clicks["action_id"]#Son todos nulls
del clicks["country_code"] #Son todos del mismo pais
del clicks["trans_id"]#Son todos valores unico
del clicks["agent_device"]#Son casi todos nulls
del clicks["brand"]#Son casi todos nulls

In [51]:
events.drop_duplicates('ref_hash', inplace = True)

In [53]:
clicks.drop_duplicates('ref_hash', inplace=True)

## Creo las ventanas

In [14]:
installs1 = installs[(installs['created'].dt.day < 21)]
installs2 = installs[(installs['created'].dt.day > 18) & (installs['created'].dt.day < 22)]
installs3 = installs[(installs['created'].dt.day > 19) & (installs['created'].dt.day < 23)]
installs4 = installs[(installs['created'].dt.day > 20) & (installs['created'].dt.day < 24)]
installs5 = installs[(installs['created'].dt.day > 21) & (installs['created'].dt.day < 25)]

In [15]:
installsCheck1 = installs[(installs['created'].dt.day > 20) & (installs['created'].dt.day < 23)]
installsCheck2 = installs[(installs['created'].dt.day > 21) & (installs['created'].dt.day < 24)]
installsCheck3 = installs[(installs['created'].dt.day > 22) & (installs['created'].dt.day < 25)]
installsCheck4 = installs[(installs['created'].dt.day > 23) & (installs['created'].dt.day < 26)]
installsCheck5 = installs[(installs['created'].dt.day > 24) & (installs['created'].dt.day < 27)]

In [16]:
installs1['sc'] = (installs1['created'] - pd.to_datetime('2019-04-18'))/np.timedelta64(1,'s')
installs2['sc'] = (installs2['created'] - pd.to_datetime('2019-04-19'))/np.timedelta64(1,'s')
installs3['sc'] = (installs3['created'] - pd.to_datetime('2019-04-20'))/np.timedelta64(1,'s')
installs4['sc'] = (installs4['created'] - pd.to_datetime('2019-04-21'))/np.timedelta64(1,'s')
installs5['sc'] = (installs5['created'] - pd.to_datetime('2019-04-22'))/np.timedelta64(1,'s')

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [17]:
installsCheck1['sc'] = (installsCheck1['created'] - pd.to_datetime('2019-04-21'))/np.timedelta64(1,'s')
installsCheck2['sc'] = (installsCheck2['created'] - pd.to_datetime('2019-04-22'))/np.timedelta64(1,'s')
installsCheck3['sc'] = (installsCheck3['created'] - pd.to_datetime('2019-04-23'))/np.timedelta64(1,'s')
installsCheck4['sc'] = (installsCheck4['created'] - pd.to_datetime('2019-04-24'))/np.timedelta64(1,'s')
installsCheck5['sc'] = (installsCheck5['created'] - pd.to_datetime('2019-04-25'))/np.timedelta64(1,'s')

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [18]:
installs1 = installs1.drop_duplicates('ref_hash', keep = 'first')
installs2 = installs2.drop_duplicates('ref_hash', keep = 'first')
installs3 = installs3.drop_duplicates('ref_hash', keep = 'first')
installs4 = installs4.drop_duplicates('ref_hash', keep = 'first')
installs5 = installs5.drop_duplicates('ref_hash', keep = 'first')
installsCheck1 = installsCheck1.drop_duplicates('ref_hash', keep = 'first')
installsCheck2 = installsCheck2.drop_duplicates('ref_hash', keep = 'first')
installsCheck3 = installsCheck3.drop_duplicates('ref_hash', keep = 'first')
installsCheck4 = installsCheck4.drop_duplicates('ref_hash', keep = 'first')
installsCheck5 = installsCheck5.drop_duplicates('ref_hash', keep = 'first')

In [19]:
del installs1['created']
del installs2['created']
del installs3['created']
del installs4['created']
del installs5['created']
del installsCheck1['created']
del installsCheck2['created']
del installsCheck3['created']
del installsCheck4['created']
del installsCheck5['created']

In [15]:
installsCheck1.head()

,application_id,ref_type,ref_hash,attributed,implicit,device_model,ip_address,device_language,sc
46742,77,1891515180541284343,40621409780134,False,False,2.019322e+18,990380024993756535,3.301378e+18,69467.657
418098,65,1891515180541284343,161514654074162,False,True,6.684966e+18,7251732924179152213,6.977049e+18,152024.805
198205,155,1891515180541284343,168103949904656,False,False,1.816356e+18,2759827436407391547,3.301378e+18,86973.188
374761,210,1891515180541284343,283297668933729,False,False,1.805456e+18,3663507817982540667,3.301378e+18,135729.476
154739,155,1891515180541284343,345999128501141,False,False,1.709140e+17,8701462141750383338,3.301378e+18,15578.532


In [20]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [21]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha =10, n_estimators = 10)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 52411.491125


## Entreno con la 2da ventana

In [22]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 52145.875450


## Entreno con la 3ra ventana

In [23]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 52041.026910


## Entreno con la 4ta ventana

In [24]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.4, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 53380.050178


## Entreno con la 5ta ventana

In [25]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.1, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 51423.175937


## Predicciones con el target

In [39]:
target = pd.read_csv('../datos/target_competencia_ids.csv')

In [40]:
targets_sc = target[target['ref_hash'].str.contains('_sc')]

In [41]:
targets_st = target[target['ref_hash'].str.contains('_st')]

In [42]:
targets_sc['ref_hash'] = targets_sc['ref_hash'].map(lambda x: str(x)[:-3])
targets_sc['ref_hash'] = targets_sc['ref_hash'].astype(np.int64)
targets_sc.dtypes

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


ref_hash    int64
obj         int64
dtype: object

In [71]:
clicks.head()

,advertiser_id,source_id,created,latitude,longitude,wifi_connection,carrier_id,os_minor,os_major,specs_brand,timeToClick,touchX,touchY,ref_type,ref_hash
0,1,2,2019-04-18T05:27:42.197Z,1.714547,0.871535,False,3.0,1.517644e+18,5.131616e+18,71913840936116953,2.317,0.968,0.503,1891515180541284343,1293710398598742392
1,1,1,2019-04-18T05:27:03.164Z,1.714512,0.871062,True,2.0,1.288578e+18,3.908390e+18,3576558787748411622,7.653,0.712,1.689,1891515180541284343,1663930990551616564
2,1,1,2019-04-18T05:42:07.926Z,1.714547,0.871535,True,4.0,2.238736e+18,3.581233e+18,3576558787748411622,464.796,0.227,0.251,1891515180541284343,8488038938665586188
3,1,1,2019-04-18T05:26:04.446Z,1.708041,0.870772,True,1.0,2.411640e+18,3.908390e+18,3576558787748411622,225.311,0.696,6.587,1891515180541284343,6488361690105189959
4,1,1,2019-04-18T05:23:37.764Z,1.715514,0.870772,True,2.0,1.517644e+18,5.131616e+18,3576558787748411622,84.736,0.059,0.142,1891515180541284343,1348993302102753419


In [43]:
installs_sin_dup = installs.drop_duplicates('ref_hash')

In [75]:
targets_sc_with_installs = targets_sc.merge(installs_sin_dup, how = 'left', on = 'ref_hash')

In [45]:
nulls = targets_sc_with_installs[targets_sc_with_installs['created'].isnull()]

In [74]:
nulls['application_id'] = nulls['ref_hash'].map(events.set_index('ref_hash')['application_id'])
nulls['ref_type'] = nulls['ref_hash'].map(events.set_index('ref_hash')['ref_type'])
nulls['device_model'] = nulls['ref_hash'].map(events.set_index('ref_hash')['device_model'])
nulls['device_language'] = nulls['ref_hash'].map(events.set_index('ref_hash')['device_language'])
nulls

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,ref_hash,obj,created,application_id,ref_type,attributed,implicit,device_model,ip_address,device_language
26,104412862089502190,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,1069066119870904979,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,1092578651697005865,0,NaT,268.0,1.494519e+18,NaN,NaN,5.544094e+18,NaN,3.3013777759776993e+18
76,11222455493472018,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1128648311587971120,0,NaT,121.0,1.891515e+18,NaN,NaN,4.898310e+18,NaN,6.977049253562486e+18
85,1136471390576544145,0,NaT,26.0,1.494519e+18,NaN,NaN,7.853392e+18,NaN,3.3013777759776993e+18
91,1156275031468463136,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,1156694545460335017,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,1251678112439909237,0,NaT,121.0,1.891515e+18,NaN,NaN,5.007865e+18,NaN,6.977049253562486e+18
163,1302737911806151146,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
targets_sc_with_installs.update(nulls)
targets_sc_with_installs.loc[53:]

,ref_hash,obj,created,application_id,ref_type,attributed,implicit,device_model,ip_address,device_language
53,1.092579e+18,0.000000,NaT,268.0,1.494519e+18,NaN,NaN,5.544094e+18,NaN,3.3013777759776993e+18
54,1.093896e+17,83899.390625,2019-04-26 19:17:18.408,26.0,1.494519e+18,False,False,8.573498e+18,6.851462e+17,3.30138e+18
55,1.095131e+18,84803.390625,2019-04-26 00:45:38.183,36.0,1.891515e+18,False,False,7.021068e+18,6.633760e+18,6.97705e+18
56,1.097691e+18,85606.031250,2019-04-26 22:07:06.954,133.0,1.891515e+18,False,True,6.871161e+18,1.240432e+18,6.97705e+18
57,1.098334e+18,83998.226562,2019-04-24 04:27:51.320,161.0,1.891515e+18,False,True,9.176732e+18,2.793880e+18,6.97705e+18
58,1.098699e+18,83596.695312,2019-04-18 04:41:25.733,86.0,1.891515e+18,False,False,7.207282e+18,5.025054e+17,3.30138e+18
59,1.100695e+18,83780.453125,2019-04-26 14:40:15.200,94.0,1.891515e+18,False,False,9.169938e+18,6.469591e+18,3.30138e+18
60,1.102680e+18,84474.789062,2019-04-25 06:30:19.854,302.0,1.891515e+18,False,False,6.415096e+18,5.553554e+18,6.97705e+18
61,1.106972e+18,85194.937500,2019-04-18 11:28:41.085,210.0,1.891515e+18,False,False,5.186987e+18,9.019540e+18,3.30138e+18
62,1.108572e+18,82596.125000,2019-04-18 21:31:44.571,122.0,1.891515e+18,False,False,4.501566e+18,4.221213e+18,3.30138e+18


In [77]:
targets_sc_with_installs.fillna({'application_id' : 0, 'ref_type' : 0, 'attributed' : False, 'implicit': False, 'device_model' : 0, 'device_language' : 0, 'ip_address' : 0}, inplace = True)

In [83]:
X = targets_sc_with_installs.loc[:,['application_id', 'ref_type', 'ref_hash', 'attributed', 'implicit', 'device_model', 'ip_address', 'device_language']]
X['device_language'] = X['device_language'].astype('float64')

In [84]:
predictions_sc = xg_reg.predict(X)

In [85]:
predictions_sc

array([82596.125, 84608.04 , 84058.08 , ..., 83780.45 , 84943.55 ,
       84608.04 ], dtype=float32)

In [86]:
targets_sc['obj'] = predictions_sc

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [87]:
predicciones_target_sc = target[target['ref_hash'].str.contains('_sc')]
predicciones_target_sc['obj'] = predictions_sc

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [91]:
target.update(predicciones_target_sc)
target

,ref_hash,obj
0,1000169251625791246_sc,82596.125000
1,1000169251625791246_st,82655.695000
2,1000395625957344683_sc,84608.039062
3,1000395625957344683_st,79631.730000
4,1003027494996471685_sc,84058.078125
5,1003027494996471685_st,82726.280000
6,1006670001679961544_sc,83874.781250
7,1006670001679961544_st,82651.010000
8,1007573308966476713_sc,83920.609375
9,1007573308966476713_st,79631.730000


In [115]:
sc_pred = pd.read_csv('../datos/PrediccionesKNN-RF-XGBoost.txt', header = None)
sc_pred.index = np.arange(1, 8074, 2)
sc_pred

,0
1,79631.730
3,79679.830
5,82655.695
7,79636.414
9,79631.730
11,82655.695
13,82726.280
15,82651.010
17,82651.010
19,79631.730


In [117]:
targets_st['obj'] = sc_pred[0]
target.update(targets_st)
target

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ref_hash,obj
0,1000169251625791246_sc,82596.125000
1,1000169251625791246_st,79631.730000
2,1000395625957344683_sc,84608.039062
3,1000395625957344683_st,79679.830000
4,1003027494996471685_sc,84058.078125
5,1003027494996471685_st,82655.695000
6,1006670001679961544_sc,83874.781250
7,1006670001679961544_st,79636.414000
8,1007573308966476713_sc,83920.609375
9,1007573308966476713_st,79631.730000


In [118]:
target.to_csv('../datos/preds-xgboost-rf-knn-nuevos-features.csv', index = False)

# Pruebo ahora con random forests

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

### Primera ventana

In [158]:
rf = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=5, min_samples_split=2, 
                           min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                           max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                           bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0,
                           warm_start=False)
rf.fit(installs1.iloc[:,:-1], installs1.iloc[:,-1])
preds = rf.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 66230.414216


### Segunda ventana

In [159]:
rf.fit(installs2.iloc[:,:-1], installs2.iloc[:,-1])
preds = rf.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67857.806283


### Tercera ventana

In [160]:
rf.fit(installs3.iloc[:,:-1], installs3.iloc[:,-1])
preds = rf.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 68837.728128


In [161]:
rf.fit(installs4.iloc[:,:-1], installs4.iloc[:,-1])
preds = rf.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67603.771269


In [162]:
rf.fit(installs5.iloc[:,:-1], installs5.iloc[:,-1])
preds = rf.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 64888.607982


# KNN

In [83]:
from sklearn.neighbors import KNeighborsRegressor

In [163]:
X, y = installs1.iloc[:,:-1],installs1.iloc[:,-1]
neigh = KNeighborsRegressor(n_neighbors=51)
neigh.fit(X, y)
preds = neigh.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 66536.923022


In [164]:
X, y = installs2.iloc[:,:-1],installs2.iloc[:,-1]
neigh.fit(X, y)
preds = neigh.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 68725.005076


In [165]:
X, y = installs3.iloc[:,:-1],installs3.iloc[:,-1]
neigh = KNeighborsRegressor(n_neighbors=51)
neigh.fit(X, y)
preds = neigh.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 69369.566275


In [166]:
X, y = installs4.iloc[:,:-1],installs4.iloc[:,-1]
neigh = KNeighborsRegressor(n_neighbors=51)
neigh.fit(X, y)
preds = neigh.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 68222.268868


In [167]:
X, y = installs5.iloc[:,:-1],installs5.iloc[:,-1]
neigh = KNeighborsRegressor(n_neighbors=51)
neigh.fit(X, y)
preds = neigh.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 65369.603274
